# Sensitivity to minimum yield constraints #

### File to generate Fig S7 ###

In [2]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso

mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

num_cores = multiprocessing.cpu_count()
model = cameo.models.bigg.iJO1366

Academic license - for non-commercial use only


In [3]:
metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol<br>3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxy<br>propanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxy-<br>acetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthio-<br>ribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}
data = {'names':[metabolite_names_dict[product] for product in carbon_dict.keys()], 'number':list(carbon_dict.values())}
metabolite_df = pd.DataFrame(data,index=list(carbon_dict.keys()))
metabolite_order = metabolite_df.sort_values(by=['number','names']).names.values

In [4]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [5]:
def yield_constraint_productivity_calculator(pecaso_template, constraint):
    temp = deepcopy(pecaso_template)
    pecaso_dict = {}
    temp.settings.yield_constraint = constraint*max(temp.production_envelope.yield_ub)
    temp.calculate_fermentation_characteristics()
    
    pecaso_dict['constraint'] = constraint
    
    if temp.two_stage_best_batch:
        pecaso_dict['ts_best'] = {}
        pecaso_dict['ts_best']['batch_productivity'] = temp.two_stage_best_batch.batch_productivity
        pecaso_dict['ts_best']['batch_yield'] = temp.two_stage_best_batch.batch_yield
        pecaso_dict['ts_best']['batch_titer'] = temp.two_stage_best_batch.batch_titer
        pecaso_dict['ts_best']['optimal_switch_time'] = temp.two_stage_best_batch.optimal_switch_time
        pecaso_dict['ts_best']['stage_one_factor'] = temp.two_stage_best_batch.stage_one_factor
        pecaso_dict['ts_best']['stage_two_factor'] = temp.two_stage_best_batch.stage_two_factor
        pecaso_dict['ts_best']['objective_value'] = temp.two_stage_best_batch.objective_value
    else:
        pecaso_dict['ts_best'] = None

    if temp.one_stage_best_batch:
        pecaso_dict['os_best'] = {}
        pecaso_dict['os_best']['batch_productivity'] = temp.one_stage_best_batch.batch_productivity
        pecaso_dict['os_best']['batch_yield'] = temp.one_stage_best_batch.batch_yield
        pecaso_dict['os_best']['batch_titer'] = temp.one_stage_best_batch.batch_titer
        pecaso_dict['os_best']['optimal_switch_time'] = temp.one_stage_best_batch.optimal_switch_time
        pecaso_dict['os_best']['stage_one_factor'] = temp.one_stage_best_batch.stage_one_factor
        pecaso_dict['os_best']['stage_two_factor'] = temp.one_stage_best_batch.stage_two_factor
        pecaso_dict['os_best']['objective_value'] = temp.one_stage_best_batch.objective_value
    else:
        pecaso_dict['os_best'] = None
    
    if temp.two_stage_suboptimal_batch:
        pecaso_dict['ts_sub'] = {}
        pecaso_dict['ts_sub']['batch_productivity'] = temp.two_stage_suboptimal_batch.batch_productivity
        pecaso_dict['ts_sub']['batch_yield'] = temp.two_stage_suboptimal_batch.batch_yield
        pecaso_dict['ts_sub']['batch_titer'] = temp.two_stage_suboptimal_batch.batch_titer
        pecaso_dict['ts_sub']['optimal_switch_time'] = temp.two_stage_suboptimal_batch.optimal_switch_time
        pecaso_dict['ts_sub']['stage_one_factor'] = temp.two_stage_suboptimal_batch.stage_one_factor
        pecaso_dict['ts_sub']['stage_two_factor'] = temp.two_stage_suboptimal_batch.stage_two_factor
        pecaso_dict['ts_sub']['objective_value'] = temp.two_stage_suboptimal_batch.objective_value
    else:
        pecaso_dict['ts_sub'] = None

    return pecaso_dict

# Substrate Uptake: Logistic #
# Objective: Productivity #
### Constrained Metric: Yield ##


In [6]:
pickling_off = open("yield_constrained_production_log.pickle","rb")
pecaso_dict = pickle.load(pickling_off)
pickling_off.close()

In [7]:
results_dict = {}
for product in pecaso_dict.keys():
    results_dict[product] = {}
    ts_best_productivities = [pecaso_dict[product][constraint]['ts_best']['batch_productivity']
                              if pecaso_dict[product][constraint]['ts_best'] else np.nan for constraint in pecaso_dict[product].keys()]


    ts_best_yields = [pecaso_dict[product][constraint]['ts_best']['batch_yield']
                      if pecaso_dict[product][constraint]['ts_best'] else np.nan for constraint in pecaso_dict[product].keys()]

    constraints_yield = list(pecaso_dict[product].keys())

    os_best_productivities = [pecaso_dict[product][constraint]['os_best']['batch_productivity']
                              if pecaso_dict[product][constraint]['os_best'] else np.nan for constraint in pecaso_dict[product].keys()]

    os_best_yields = [pecaso_dict[product][constraint]['os_best']['batch_yield'] 
                      if pecaso_dict[product][constraint]['os_best'] else np.nan for constraint in pecaso_dict[product].keys()]



    ts_sub_productivities = [pecaso_dict[product][constraint]['ts_sub']['batch_productivity']
                             if pecaso_dict[product][constraint]['ts_sub']
                             else np.nan for constraint in pecaso_dict[product].keys()]

    ts_sub_yields = [pecaso_dict[product][constraint]['ts_sub']['batch_yield']
                     if pecaso_dict[product][constraint]['ts_sub']
                     else np.nan for constraint in pecaso_dict[product].keys()]


    max_yield = max(os_best_yields)
    molar_mass = [metabolite for metabolite in model.metabolites if metabolite.name==product][0].formula_weight

    results_dict[product]['ts_best_productivities'] = np.array(ts_best_productivities)*molar_mass/1000
    results_dict[product]['ts_best_yields'] = np.array(ts_best_yields)/max_yield*100
    results_dict[product]['os_best_productivities'] = np.array(os_best_productivities)*molar_mass/1000
    results_dict[product]['os_best_yields'] = np.array(os_best_yields)/max_yield*100
    results_dict[product]['ts_sub_productivities'] = np.array(ts_sub_productivities)*molar_mass/1000
    results_dict[product]['ts_sub_yields'] = np.array(ts_sub_yields)/max_yield*100
    results_dict[product]['constraints_yield'] = np.array(constraints_yield)*100

In [14]:
for k,carbon_number in enumerate(sorted(set(metabolite_df.number.values))):
    print("No. of carbons: ", carbon_number)
    metabolite_list = metabolite_df[metabolite_df.number==carbon_number].sort_values(by='names').index
    num_rows = int(len(metabolite_list)/6-0.1)+1
    titles = [metabolite_names_dict[metabolite] for metabolite in metabolite_list]
    fig = subplots.make_subplots(rows=num_rows, cols=6, subplot_titles=titles,print_grid=False, horizontal_spacing=0.04)
    for i,product in enumerate(metabolite_list):
        
        fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                    y=results_dict[product]['ts_best_productivities'],
                                    name='Optimized Two Stage',
                                    legendgroup='ts_best', line={'color': '#8DC447'},
                                    showlegend=False), row=int(i/6)+1, col=int(i%6)+1)

        fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                    y=results_dict[product]['ts_sub_productivities'],
                                    name='Traditional Two Stage',
                                    legendgroup='ts_sub', line={'color': '#FF5555'},
                                    showlegend=False), row=int(i/6)+1, col=int(i%6)+1)

        fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                    y=results_dict[product]['os_best_productivities'],
                                    name='Best One Stage',
                                    legendgroup='os_best', line={'color': '#76D1F4'},
                                    showlegend=False), row=int(i/6)+1, col=int(i%6)+1)



        fig.update_yaxes(range=[0, int((max(results_dict[product]['ts_best_productivities'])+0.1)*2)/2+0.5],
                         row=int(i/6)+1, col=int(i%6)+1)

        if i%6==0:
            fig.update_yaxes(title_text='Max Rate(g/L.h)',
                             title_font=dict(size=12, color='black'),
                             title_standoff=0,
                             automargin=True, row=int(i/6)+1, col=int(i%6)+1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_text='Min Yield (% Max)',
                     title_font=dict(size=12, color='black'),
                     tickfont=dict(size=14, color='black'),
                     title_standoff=0,
                     automargin=True,
                     ticks='outside',
                     nticks=5)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     tickangle=0,
                     ticks='outside',
                     tickfont=dict(size=14, color='black'))
    fig.update_layout(width=1000,
                      height=90+210*num_rows,
                      margin={'l':8})
    fig.update_annotations(dict(font=dict(size=16, color='black')))

    plot(fig)
    if not os.path.exists('images'):
        os.mkdir('images')
    #pio.write_image(fig, 'images/yield_constrained_productivity_objective_C_'+str(k+1)+'.svg')   


No. of carbons:  1


No. of carbons:  2


No. of carbons:  3


No. of carbons:  4


No. of carbons:  5


No. of carbons:  6


No. of carbons:  >6


In [27]:
for i,product in enumerate(['D-Lactate']):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                y=results_dict[product]['ts_best_productivities'],
                                name='Optimized Two Stage',
                                legendgroup='ts_best', line={'color': '#8DC447'},
                                showlegend=False))

    fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                y=results_dict[product]['ts_sub_productivities'],
                                name='Traditional Two Stage',
                                legendgroup='ts_sub', line={'color': '#FF5555'},
                                showlegend=False))

    fig.add_trace(go.Scatter(x=results_dict[product]['constraints_yield'], 
                                y=results_dict[product]['os_best_productivities'],
                                name='Best One Stage',
                                legendgroup='os_best', line={'color': '#76D1F4'},
                                showlegend=False))

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                 title_text='Min Yield (% Max)',
                 title_font=dict(size=18, color='black'),
                 tickfont=dict(size=18, color='black'),
                 title_standoff=0,
                 automargin=True,
                 ticks='outside',
                 nticks=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                 ticks='outside',
                 tickfont=dict(size=18, color='black'),
                 range=[0, int((max(results_dict[product]['ts_best_productivities'])+0.1)*2)/2+0.5],
                 title_text='Productivity (g/L.h)',
                 title_font=dict(size=18, color='black'),
                 title_standoff=10,
                 automargin=True)
fig.update_layout(width=350,
                  height=400,
                  margin={'l':8})

plot(fig)

if not os.path.exists('images'):
    os.mkdir('images')
#pio.write_image(fig, 'images/yield_constrained_productivity_lactate.svg')   
